## 2. Clasificación con SVM

Como se ha mencionado en el preprocesamiento, en el problema que aquí se estudia, cada individuo puede pertenecer a una clase, las dos o ninguna, por lo que se está tratando de un problema multietiqueta. Para usar una máquina de soporte vectorial en clasificación es necesario codificar la salida para que las distintas variantes de SVM pueda realizar la clasificación.

In [17]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.svm import LinearSVC, SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier



In [18]:
X_train = pd.read_csv("preprocessed_X_train.csv", index_col="respondent_id")
y_train = pd.read_csv("preprocessed_y_train.csv", index_col="respondent_id")

# Cargar el dataset de prueba
X_test = pd.read_csv("preprocessed_X_test.csv", index_col="respondent_id")
y_test = pd.read_csv("preprocessed_y_test.csv", index_col="respondent_id")

In [19]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape, y_test.shape)


Train set: (20739, 96) (20739, 2)
Test set: (5195, 96) (20739, 2)


In [20]:
X_test.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
respondent_id,,,,,,,,,,,,,,,,,,,,,
22317,1.0,1.0,0,1,0,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1215,2.0,2.0,0,1,0,1,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10540,2.0,1.0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7407,3.0,1.0,0,1,0,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14697,1.0,1.0,0,1,0,1,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X_train.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv
respondent_id,,,,,,,,,,,,,,,,,,,,,
12230,2.0,2.0,0,1,0,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8871,1.0,1.0,0,0,0,1,0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2390,1.0,1.0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9227,2.0,1.0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9064,2.0,1.0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Antes de proceder, es necesario escalar los valores, sobre todo para que las variables de mayor rango (pertenencientes a las preguntas de opinión mayoritariamente) no influyan en el cálculo de las distancias con SVM. Dentro de las posibilidades de escalado, se puede optar por:
- Normalización zero-mean: Escala los valores en un rango $[0,1]$ en función de la media y la varianza de la variable. Esta normalización se usa cuando las distribuciones de las variables siguen aproximadamente una distribución normal.
- Normalización MinMaxScaling: Escala los valores en un rango $[0,1]$ en función de los valores máximos y mínimos de cada variable. Esta técnica de escalado conserva las características de los datos originales tras la transformación. 

Dado que no se conocen las distribuciones originales de los datos, se va a optar por el uso de escalado MinMax

In [22]:
# Escalar características
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[0.25      , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.66666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.25      , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 0.66666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.25      , 0.33333333, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [23]:
# Asegurarse de que todos los valores sean numéricos
#X_train = X_train.astype(float)
#X_test = X_test.astype(float)
y_train = y_train.astype(int)

from sklearn.model_selection import train_test_split

X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(
    X_train_scaled, y_train, test_size=0.33, random_state=42
)

Una vez tenemos todos los datos en el mismo formato, podemos pasar a implementar SVM. Este algoritmo trata de sobredimensionar los datos para poder generar un hiperplano de separación entre dos clases, por lo que, de manera nativa no es capaz de realizar una clasificación multietiqueta. Para solventar este problema se usa un pequeño 'truco' a la hora de implementar el algoritmo, que es la descomposición binaria de la respuesta obtenida, de esta manera SVM puede abordar los problemas multietiqueta como problemas independientes:

- Si se entrena un clasificador para cada par de valores posibles se está usando una estrategia OVO (One-vs-One)
- Si el clasificador es capaz de distinguir en cada caso su etiqueta frente a las demás se usa una estrategia OVR (One-vs-Rest)

Otra opción para la clasificación multietiqueta pasa por el uso de la función 'MultiOutputClassifier' implementada en 'sklearn', capaz de tratar cada etiqueta como un problema binario independiente, entrenando un clasificador para cada etiqueta sin comparar con las demás:


#### Clasificación SVM con RBF y balanceo de clases

##### Optimización por GridSearch y clases balanceadas

In [24]:
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Paso 2: Configurar MultiOutputClassifier para manejar multietiqueta
multi_label_svm = MultiOutputClassifier(base_svm)

# Paso 3: Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'estimator__C': [0.1, 1, 10],           # Regularización
    'estimator__gamma': ['scale', 'auto'],  # Parámetro del kernel RBF
    'estimator__max_iter': [10000, 50000],  # Iteraciones máximas
    'estimator__kernel': ['rbf']            # Kernel
}

# Paso 4: Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=multi_label_svm,
    param_grid=param_grid,
    cv=3,  # Validación cruzada de 5 pliegues
    scoring='roc_auc',  # Métrica para evaluar el modelo
    verbose=3,  # Para ver el progreso
    n_jobs=-1,   # Usar todos los núcleos disponibles
    refit=True

)

# Paso 5: Entrenar con los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Paso 6: Resultados
print("Mejores parámetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en optimización (train): {grid_search.best_score_}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))
roc_auc = roc_auc_score(y_test, y_valid_pred, average='macro', multi_class='ovr')

# Imprimir el resultado
print(f"ROC AUC Score (test): {roc_auc:.4f}")


Fitting 3 folds for each of 12 candidates, totalling 36 fits


KeyboardInterrupt: 

#### Optimización de parámetros con RandomSearch y clases balanceadas

En este caso buscaremos los valores optimos para gamma= ['auto','scale'] con random search y veremos si los resultados obtenidos son iguales:

In [67]:
# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')

# Configurar MultiOutputClassifier
multi_label_svm = MultiOutputClassifier(base_svm)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': np.logspace(-3, 1, 5),  # Ajuste más fino para 'C'
    'estimator__gamma': ["scale", "auto"], 
    'estimator__max_iter': [10000, 50000, 100000],
    'estimator__kernel': ['rbf']
}



# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__max_iter': 50000, 'estimator__kernel': 'rbf', 'estimator__gamma': 'auto', 'estimator__C': np.float64(10.0)}
Mejor score en train: 0.8574269006524332
Accuracy en validación: 0.6626731561213028
ROC AUC Score en Test: 0.7892
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.53      0.77      0.63      1135
           1       0.77      0.78      0.77      2488

   micro avg       0.67      0.78      0.72      3623
   macro avg       0.65      0.77      0.70      3623
weighted avg       0.69      0.78      0.73      3623
 samples avg       0.36      0.38      0.36      3623



De aquí en adelante usaremos la optimización por Random search por ser más efectiva en el cálculo.

#### SVM RBF con clases Desbalanceadas

In [68]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)

# Configurar MultiOutputClassifier
multi_label_svm = MultiOutputClassifier(base_svm)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': np.logspace(-3, 1, 5),  # Ajuste más fino para 'C'
    'estimator__gamma': np.logspace(-3, 1, 5),  # Ajuste más fino para 'gamma'
    'estimator__max_iter': [10000, 50000],
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__max_iter': 10000, 'estimator__gamma': np.float64(0.01), 'estimator__C': np.float64(1.0)}
Mejor score en train: 0.8537376785855378
Accuracy en validación: 0.6767128416323475
ROC AUC Score en Test: 0.7363
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.72      0.43      0.54      1135
           1       0.78      0.74      0.76      2488

   micro avg       0.77      0.64      0.70      3623
   macro avg       0.75      0.58      0.65      3623
weighted avg       0.76      0.64      0.69      3623
 samples avg       0.34      0.32      0.33      3623



#### SVM Kernel Lineal y Balanceo de Clases

También podemos probar LinearSVC, en este caso el clasificador SVM que se usa kernel lineal y al no obtener probabilidades, no podemos calcular el roc_auc para evaluar el randomsearch:


Con balanceo de clases

In [69]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced'))

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'estimator__tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'estimator__max_iter': [10000, 20000, 50000],  # Opciones discretas
    'estimator__dual': [True, False]  # Opciones binarias
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_lsvm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)

# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__tol': 0.001, 'estimator__max_iter': 50000, 'estimator__dual': True, 'estimator__C': 0.01}
Accuracy en validación: 0.6536877573942343
ROC AUC Score en Test: 0.7860
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.53      0.77      0.63      1135
           1       0.76      0.78      0.77      2488

   micro avg       0.67      0.78      0.72      3623
   macro avg       0.64      0.77      0.70      3623
weighted avg       0.68      0.78      0.72      3623
 samples avg       0.36      0.38      0.36      3623



In [70]:
random_search.best_score_

np.float64(0.6401122330897181)

El mejor modelo,...................

#### SVM Kernel Lineal y Desbalanceo de Clases

In [71]:
from sklearn.svm import LinearSVC

multi_label_lsvm = MultiOutputClassifier(LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000))

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'estimator__C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'estimator__tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'estimator__max_iter': [10000, 20000, 50000],  # Opciones discretas
    'estimator__dual': [True, False]  # Opciones binarias
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    multi_label_lsvm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(X_train_scaled, y_train)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)

# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(X_train_scaled, y_train)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_valid_pred, zero_division=0))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Mejores parámetros: {'estimator__tol': 0.001, 'estimator__max_iter': 20000, 'estimator__dual': True, 'estimator__C': 1}
Accuracy en validación: 0.6765256458255335
ROC AUC Score en Test: 0.7412
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.70      0.46      0.56      1135
           1       0.77      0.75      0.76      2488

   micro avg       0.76      0.66      0.70      3623
   macro avg       0.74      0.60      0.66      3623
weighted avg       0.75      0.66      0.70      3623
 samples avg       0.35      0.33      0.33      3623



In [72]:
random_search.best_score_

np.float64(0.6752164149677964)

#### One Vs One y One vs Rest

Comparamos los resultados con los que se obtienen a partir de OVO y OVR:

In [73]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

ovr = OneVsRestClassifier(LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, class_weight='balanced', tol= 0.001)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_scaled, y_train)
y_pred = ovr.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_test, y_pred, zero_division=0))


Accuracy en validación: 0.6525645825533508
ROC AUC Score: 0.7859
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.53      0.77      0.63      1135
           1       0.75      0.78      0.77      2488

   micro avg       0.66      0.78      0.72      3623
   macro avg       0.64      0.78      0.70      3623
weighted avg       0.68      0.78      0.72      3623
 samples avg       0.36      0.38      0.36      3623



In [74]:
ovr = OneVsRestClassifier(SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced', C=10, gamma=0.01)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_train_scaled, y_train)
y_pred = ovr.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVR:\n", classification_report(y_test, y_pred, zero_division=0))

c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy en validación: 0.6405840509172595
ROC AUC Score: 0.7848
Reporte de clasificación OVR:
               precision    recall  f1-score   support

           0       0.51      0.79      0.62      1135
           1       0.75      0.79      0.77      2488

   micro avg       0.65      0.79      0.71      3623
   macro avg       0.63      0.79      0.69      3623
weighted avg       0.67      0.79      0.72      3623
 samples avg       0.36      0.39      0.37      3623



OVO

In [75]:
ovo = OneVsOneClassifier(LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, class_weight='balanced', tol= 0.001)); # Usamos mismos parámetros que antes y la misma semilla
ovo.fit(X_train_scaled, y_train)
y_pred = ovr.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVO:\n", classification_report(y_test, y_pred, zero_division=0))

ValueError: y should be a 1d array, got an array of shape (21365, 2) instead.

In [8]:
ovo =MultiOutputClassifier(OneVsOneClassifier(SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced', C=10, gamma=0.01))); # Usamos mismos parámetros que antes y la misma semilla
ovo.fit(X_train_scaled, y_train)
y_pred = ovo.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVO:\n", classification_report(y_test, y_pred, zero_division=0))

c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\er_ja\anaconda3\envs\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


NameError: name 'ovr' is not defined

In [ ]:
ovo =MultiOutputClassifier( OneVsOneClassifier(LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, class_weight='balanced', tol= 0.001))); # Usamos mismos parámetros que antes y la misma semilla
ovo.fit(X_train_scaled, y_train)
y_pred = ovr.predict(X_test_scaled)
roc_auc = roc_auc_score(y_test, y_pred)

print("Accuracy en validación:", accuracy_score(y_test, y_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score: {roc_auc:.4f}")
print("Reporte de clasificación OVO:\n", classification_report(y_test, y_pred, zero_division=0))

Parece claro que el método OVR con LinearSVC se acerca en mayor medida a nuestros resultados con MultiOutputClassifier. 

#### Simplificación del Problema

Como hemos mencionado anteriormente, SVM no trabaja directamente con problemas multietiqueta, por lo que se envuelve con MultiOutputClassifier. Otra opción válida es dividir inicialmente las etiquetas para descomponerlo en dos SVM distintos:

In [ ]:
y=y_train["seasonal_vaccine"]
z=y_train["h1n1_vaccine"]

#### Vacuna estacional

In [101]:
x_train=X_train_scaled
x_test= X_test_scaled

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))


##### Sin balanceo

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

#### LinearSVM con balanceo

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced')

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

#### LinearSVM Sin balanceo

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, y)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, y)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

#### Vacuna H1N1. Clases balanceadas

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000, class_weight='balanced')


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

#### Sin balanceo

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm = SVC(kernel='rbf', probability=True, random_state=42, max_iter=10000)


# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': np.logspace(-2, 1, 4),        # Rango reducido para 'C'
    'gamma': np.logspace(-2, 1, 4),    # Rango reducido para 'gamma'
    'max_iter': [10000, 50000],        # Iteraciones máximas
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

#### LinearSVC balanceado

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000, class_weight='balanced')

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

#### Sin Balanceo

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

# Definir el modelo base
base_svm =LinearSVC(C=1.0, random_state=42, dual=True, max_iter=10000)

# Definir el espacio de búsqueda de parámetros
param_dist = {
    'C': [0.01, 0.1, 1, 10],  # Valores discretos para C
    'tol': [ 1e-3, 1e-2],  # Tolerancia fija en valores comunes
    'max_iter': [10000, 20000, 50000],  # Opciones discretas
    'dual': [True, False]  # Opciones binarias
}


# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    base_svm,
    param_distributions=param_dist,
    n_iter=20,  # Número de iteraciones aleatorias
    cv=3,       # Validación cruzada
    scoring='roc_auc',  # Métrica de evaluación
    n_jobs=-1,  # Usar todos los núcleos disponibles
    verbose=3,  # Progreso
    random_state=42,
    refit=True
)

# Ejecutar RandomizedSearchCV
random_search.fit(x_train, z)

# Imprimir los mejores parámetros y el score
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score en train:", random_search.best_score_)


# Ajustamos el modelo y lo evaluamos
best_model = random_search.best_estimator_
best_model.fit(x_train, z)
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_valid_pred = best_model.predict(x_test)
roc_auc = roc_auc_score(y_test["seasonal_vaccine"], y_valid_pred)

print("Accuracy en validación:", accuracy_score(y_test["seasonal_vaccine"], y_valid_pred))
# Imprimir el roc_auc_score global
print(f"ROC AUC Score en Test: {roc_auc:.4f}")
print("Reporte de clasificación:\n", classification_report(y_test["seasonal_vaccine"], y_valid_pred, zero_division=0))

# Ensembles


### Para LinearSVM con Balanceo entre clases

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.multioutput import MultiOutputClassifier


# 1. Modelo base SVM con MultiOutputClassifier para multietiqueta
base_svm = LinearSVC(C=0.1, random_state=42, dual=True, max_iter=10000, tol=0.001, class_weight='balanced')

# 2. Bagging clásico con SVM
bagging_svm = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=0.8,   # Usa el 80% de las muestras
    max_features=1.0,  # Usa el 100% de las características
    bootstrap=True,
    random_state=42
)
# Envolvemos el bagging con MultiOutputClassifier
bagging_svm_multi = MultiOutputClassifier(bagging_svm)


# 3. Random Subspaces (Bagging con muestreo de características)
random_subspaces = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=1.0,   # Usa el 100% de las muestras
    max_features=0.5,  # Usa el 50% de las características
    bootstrap=False,
    random_state=42
)
# Envolvemos el Random Subspaces con MultiOutputClassifier
random_subspaces_multi = MultiOutputClassifier(random_subspaces)

# Lista de modelos para comparar
models = {
    "Bagging_SVM": bagging_svm_multi,
    "Random_Subspaces": random_subspaces_multi
}

# Entrenamiento y evaluación
results = {}
for name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train_scaled, y_train)

    # Predicción
    y_pred = model.predict(X_test_scaled)

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Guardar resultados
    results[name] = {"accuracy": accuracy, "report": report}
    print(f"=== {name} ===")
    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(report)

#### Ensemble de SVM con kernel RBF

In [ ]:

# 1. Modelo base SVM con MultiOutputClassifier para multietiqueta
base_svm = SVC(kernel='rbf', C=10, random_state=42,  max_iter=10000, gamma=0.01, class_weight='balanced')

# 2. Bagging clásico con SVM
bagging_svm = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=0.8,   # Usa el 80% de las muestras
    max_features=1.0,  # Usa el 100% de las características
    bootstrap=True,
    random_state=42
)
# Envolvemos el bagging con MultiOutputClassifier
bagging_svm_multi = MultiOutputClassifier(bagging_svm)


# 3. Random Subspaces (Bagging con muestreo de características)
random_subspaces = BaggingClassifier(
    estimator=base_svm,
    n_estimators=10,  # 10 modelos base
    max_samples=1.0,   # Usa el 100% de las muestras
    max_features=0.5,  # Usa el 50% de las características
    bootstrap=False,
    random_state=42
)
# Envolvemos el Random Subspaces con MultiOutputClassifier
random_subspaces_multi = MultiOutputClassifier(random_subspaces)

# Lista de modelos para comparar
models = {
    "Bagging_SVM": bagging_svm_multi,
    "Random_Subspaces": random_subspaces_multi
}

# Entrenamiento y evaluación
results = {}
for name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train_scaled, y_train)

    # Predicción
    y_pred = model.predict(X_test_scaled)

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Guardar resultados
    results[name] = {"accuracy": accuracy, "roc_auc":roc_auc , "report": report}
    print(f"=== {name} ===")
    print(f"Accuracy: {accuracy}")
    print(f"Roc-AUC: {roc_auc}")
    print("Classification Report:")
    print(report)